In [1]:
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import MinMaxScaler


In [2]:
#{"DX": [], "DY": [], "DZ": [], "H": [], "D1": [], "D2": [], "log_f": [], "RP": []})
DATA = pd.read_csv("../csv/data.csv")
DATA.pop("Unnamed: 0")
DATA.pop("D1")
DATA.pop("D2")

ROW_NUM = len(DATA.index)
TRAINING_NUM = int(ROW_NUM / 5 * 3)
VALIDATION_NUM = int(ROW_NUM / 5 )

TRAINING_DATA = DATA.iloc[ :TRAINING_NUM,:]
VALIDATION_DATA = DATA.iloc[TRAINING_NUM:TRAINING_NUM+VALIDATION_NUM,:]
TEST_DATA = DATA.iloc[TRAINING_NUM+VALIDATION_NUM:,:]
TRAINING_VAL = pd.DataFrame(TRAINING_DATA.pop('RP'))
VALIDATION_VAL = pd.DataFrame(VALIDATION_DATA.pop('RP'))
TEST_VAL = pd.DataFrame(TEST_DATA.pop('RP'))


In [3]:
def scaling(DATA):
    min_max_scaler = MinMaxScaler()
    fitted = min_max_scaler.fit(DATA)
    DATA = min_max_scaler.transform(DATA)
    return pd.DataFrame(DATA)
    
TRAINING_DATA = scaling(TRAINING_DATA)
VALIDATION_DATA = scaling(VALIDATION_DATA)
TEST_DATA = scaling(TEST_DATA)

In [4]:
model = keras.models.Sequential([
    keras.layers.Dense(8,activation="sigmoid", input_dim=7),
    keras.layers.Dense(1, activation="sigmoid")
])

model.compile(loss="mean_squared_error", optimizer='sgd')


2022-05-10 16:37:07.726382: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
model.fit(TRAINING_DATA, TRAINING_VAL, epochs=100, batch_size=100, validation_data=(VALIDATION_DATA, VALIDATION_VAL))


Epoch 1/100
5/5 [==============================] - 0s 27ms/step - loss: 0.0598 - val_loss: 0.0647
Epoch 2/100
5/5 [==============================] - 0s 4ms/step - loss: 0.0596 - val_loss: 0.0645
Epoch 3/100
5/5 [==============================] - 0s 8ms/step - loss: 0.0594 - val_loss: 0.0644
Epoch 4/100
5/5 [==============================] - 0s 7ms/step - loss: 0.0592 - val_loss: 0.0642
Epoch 5/100
5/5 [==============================] - 0s 6ms/step - loss: 0.0590 - val_loss: 0.0641
Epoch 6/100
5/5 [==============================] - 0s 6ms/step - loss: 0.0588 - val_loss: 0.0640
Epoch 7/100
5/5 [==============================] - 0s 11ms/step - loss: 0.0587 - val_loss: 0.0639
Epoch 8/100
5/5 [==============================] - 0s 12ms/step - loss: 0.0585 - val_loss: 0.0638
Epoch 9/100
5/5 [==============================] - 0s 5ms/step - loss: 0.0584 - val_loss: 0.0637
Epoch 10/100
5/5 [==============================] - 0s 6ms/step - loss: 0.0583 - val_loss: 0.0636
Epoch 11/100
5/5 [========

In [6]:

scores = model.evaluate(TEST_DATA, TEST_VAL, batch_size=20)
print(scores)


9/9 [==============================] - 0s 783us/step - loss: 0.0609
0.06087382137775421


In [7]:
predictions = pd.DataFrame(model.predict(TEST_DATA))
predictions.columns=["PREDICITONS"]
TEST_VAL.columns=["TEST_VAL"]
result=pd.concat([predictions, TEST_VAL], axis=1)
result.iloc[:30,:].to_csv("4-8-1.csv")
